Applying Optimization and Grid Search

In [1]:
#imports
import itertools
import pandas as pd

# Genetic Algorithm
from library.GA import genetic_algorithm, get_best_individual 

# Class GA Solution
from library.lineup import LUGASolution

# Mutation Functions
from library.mutation_funcs import block_rotation_mutation, two_phase_shuffle_mutation, semi_shuffle

# Crossover Functions
from library.xo_funcs import cyclic_crossover, custom_pmxo

# Selection Algorithm
from library.selection_algorithms import tournament_selection, ranking_selection
from library.selection_algorithms import fitness_proportionate_selection #NÃO PODEMOS USAR ERA SÓ PARA EXEMPLO

# Line-up Problem: Datasets
from library.lineup import artists_df, conflicts_df

In [4]:
POP_SIZE = 10

initial_population = [LUGASolution(mutation_function = block_rotation_mutation, crossover_function = cyclic_crossover
                                   )
                      for i in range(POP_SIZE)]

genetic_algorithm(initial_population=initial_population,
                  max_gen=10,
                  selection_algorithm = tournament_selection,
                  maximization=True,
                  xo_prob=0.8,
                  mut_prob=1,
                  elitism=True,
                  verbose=False)[0]

#get_best_ind(initial_population, maximization=True)

Generations:   0%|          | 0/10 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/10 [00:00<?, ?ind/s]

<LUSolution (5 Stages, 7 Slots)>
╒═════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤════════════════╤════════════════════╤══════════════════╤════════════════════╕
│         │       Slot 1        │          Slot 2          │        Slot 3         │     Slot 4     │       Slot 5       │      Slot 6      │       Slot 7       │
╞═════════╪═════════════════════╪══════════════════════════╪═══════════════════════╪════════════════╪════════════════════╪══════════════════╪════════════════════╡
│ Stage 1 │   The Jazz Nomads   │     Synthwave Saints     │   Celestial Voyage    │  Velvet Pulse  │    Turbo Vortex    │ Nightfall Sonata │    Aurora Skies    │
├─────────┼─────────────────────┼──────────────────────────┼───────────────────────┼────────────────┼────────────────────┼──────────────────┼────────────────────┤
│ Stage 2 │   Crimson Harmony   │ The Polyrhythm Syndicate │   Cosmic Frequency    │ Midnight Echo  │ Parallel Dimension │   Neon Reverie   │   Lunar Sp

In [5]:
def comb_tuning(mut_func: dict, xo_func: dict, selection_algo: list,
                n_runs: int = 30, max_gen:int = 100, pop_size: int = 50):
    
    """
    Returns a DataFrame with the following columns:
    - 'Combination' - Combination tuple of a mutation, a crossover and a selection algorithm
    - 'Run' - Each Combination is runned a specified amount of times, this column specifies the run 
    number in which the results were obtained.
    - 'Generation' - Each Run is composed of a specified amount of generations, this column specifies the 
    generation number in which the results were obtained.
    - 'Best Fitness' - This is the best fitness obtained at each generation.

    Purpose:
    With this df we can play around inter-run and intra-run statistics, since it contains 
    all the results from the various combinations
    """

    combinations = list(itertools.product(list(mut_func.keys()), list(xo_func.keys()), selection_algo))

    combs_results = []
    for comb in combinations:
        # Store the best results of each generation for each run.
        runs_results = []
        for run in range(1, n_runs+1):
            initial_population = [LUGASolution(mutation_function = comb[0], crossover_function = comb[1])
                                               for i in range(pop_size)]
            
            _, generation_df = genetic_algorithm(initial_population=initial_population,
                    max_gen=max_gen,
                    selection_algorithm = comb[2],
                    maximization=False,
                    xo_prob=xo_func[comb[1]],
                    mut_prob= mut_func[comb[0]],
                    elitism=True,
                    verbose=False)
            
            current_run = pd.DataFrame()
            current_run['Run'] = [run for _ in range(1, n_runs+1)] #fill column 'Run' with the #run it is in
            
            #with run=#run, merge the df that contains the columns with #generation and its best fitness
            one_run = pd.concat([current_run, generation_df], axis=1) 
            runs_results.append(one_run)
        
        current_comb_all_runs = pd.concat(runs_results) #concat all the different runs along the index axis
        
        # Generate filename based on combination
        filepath = f"../combination_search/comb_{comb[0]}_{comb[1]}_{comb[2]}.csv"
        # Save to CSV
        current_comb_all_runs.to_csv(filepath, index=False)
        
        current_comb = pd.DataFrame()
        current_comb['Combination'] = [(comb[0], comb[1], comb[2]) for _ in range(len(combinations))]
        one_comb = pd.concat([current_comb, current_comb_all_runs], axis=1)
        combs_results.append(one_comb)
    
    final_results = pd.concat(combs_results)
    final_results.to_csv(f"../combination_search/final_results.csv", index=False)
    
    return final_results

In [8]:
#Example
mut_func = {block_rotation_mutation:0.07, two_phase_shuffle_mutation: 0.1, semi_shuffle: 0.2}
xo_func = {cyclic_crossover:0.7, custom_pmxo: 0.8}
selection_algo = [tournament_selection, ranking_selection]

In [7]:
#experimentar 1ª com poucas runs e poucas gen e pop baixa para ver se corre tudo bem
comb_tuning(mut_func = mut_func, xo_func = xo_func, selection_algo=selection_algo,
            n_runs=3, max_gen=10, pop_size=20)

Generations:   0%|          | 0/10 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/20 [00:00<?, ?ind/s]

Generations:   0%|          | 0/10 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/20 [00:00<?, ?ind/s]

Generations:   0%|          | 0/10 [00:00<?, ?gen/s]

Gen  1:   0%|          | 0/20 [00:00<?, ?ind/s]

OSError: Cannot save file into a non-existent directory: '..\combination_search'